# Data processing for Force Estimation

In [1]:
import numpy as np
import cv2
import cv2.aruco as aruco
from IPython.display import display, clear_output
import os
import glob
from scipy import interpolate
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt

%matplotlib inline

## Plan for video processing:
1. Retrieve images from video.
2. Manually delete frames outside the experiment.

In [ ]:
def video2images(video_file_path, output_path, verbose=False):
    # create directory for output
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    cap = cv2.VideoCapture(video_file_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    resolution = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    N = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    
    n = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:
            cv2.imwrite(os.path.join(output_path, '{:04d}.png'.format(n)), frame)

            n += 1
            if verbose and n % int(fps) == 0:
                display("Progress: {:.1f}% {}".format(100.0 * n / N, output_path[-14:-8]))
                clear_output(wait=True)

        else:
            break
            
    cap.release()
    
    if verbose:
        print("Retrieved frames forom {}".format(video_file_path))
        print("Stored images in {}".format(output_path))
        print("fps: {}".format(fps))
        print("number of frames: {}".format(N))
        print("resolution: {}".format(resolution))

In [ ]:
# check fps
for subject_number in range(1, 3):
    for experiment_number in range(1, 11):
        video_file_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/original.MP4'.format(subject_number, experiment_number)
        cap = cv2.VideoCapture(video_file_path)
        cap.release()

In [ ]:
for subject_number in range(1, 3):
    for experiment_number in range(1, 11):
        video_file_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/original.MP4'.format(subject_number, experiment_number)
        output_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
        video2images(video_file_path, output_path, verbose=True)

#### Bash commands to delete frames outside experiments
1. xdg-open 0000.png
2. for i in {0000..0005}; do rm $i.png; done


In [ ]:
# shift filenames to start with 0
def rename_files(path, verbose=False):
    files = glob.glob('{}/*.png'.format(path))[::-1]
    files.sort()
    delta = int(files[0][-8:-4])
    n = len(files)
    for i in range(n):
        new_name = path + '{:04d}.png'.format(int(files[i][-8:-4]) - delta)
        os.rename(files[i], new_name)
#         if verbose and i % 100 == 0:
#             display("Progress: {:.1f}% {}".format(100.0 * i / n, path[-14:-7]))
#             clear_output(wait=True)

In [ ]:
for subject_number in range(1, 2):
    for experiment_number in range(1, 11):
        path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
        rename_files(path, verbose=True)

## Plan for force data processing:
1. Apply Butterworth low-pass filter
2. 

In [ ]:
class ButterLowpasssFilter:
    def __init__(self, cutoff=3, fs=60.0, order=1):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        self.fs = fs
        self.b, self.a = butter(order, normal_cutoff, btype='low', analog=False)

    def filter(self, data):
        y = filtfilt(self.b, self.a, data, axis=0)
        return y

    def get_t(self, n):
        return n/self.fs

filt = ButterLowpasssFilter()

In [ ]:
def get_force_data(frames_path, force_data_path, frame_rate=60.0, interpolated=True):
    # get frame indexes
    frames = glob.glob('{}*.png'.format(frames_path))[::-1]
#     frame_indexes = np.array([int(x[-8:-4]) for x in frames])
#     frame_indexes.sort()

    # read force measurements
    force_data = np.loadtxt(force_data_path, delimiter=',')
    if not interpolate:
        return force_data[:,0], force_data[:,1], force_data[:,2] # return raw data

    # match frames with the nearest measurements
#     t_frame = frame_indexes * 1000.0 / frame_rate # in milliseconds
    t_frame = np.linspace(0.0, 30000.0, len(frames)) # in milliseconds
    f1 = interpolate.interp1d(force_data[:,0], force_data[:,1], kind='nearest', bounds_error=False, fill_value='extrapolate')
    f2 = interpolate.interp1d(force_data[:,0], force_data[:,2], kind='nearest', bounds_error=False, fill_value='extrapolate')
    vf1 = np.vectorize(f1)
    vf2 = np.vectorize(f2)
    force1_interpolated = vf1(t_frame)
    force2_interpolated = vf2(t_frame)
    return t_frame, force1_interpolated, force2_interpolated

In [ ]:
subject_number = 1
experiment_number = 1
frames_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
force_data_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/raw_force_data.txt'.format(subject_number, experiment_number)
t, f1, f2 = get_force_data(frames_path, force_data_path)
f1_filtered = filt.filter(f1)
f2_filtered = filt.filter(f2)

t_raw, f1_raw, f2_raw = get_force_data(frames_path, force_data_path, interpolated=False)

In [ ]:
plt.figure(figsize=(20,15))
plt.plot(t, f1, 'r-', linewidth=3, label='f1')
# plt.plot(t, f2, 'b-', linewidth=2, label='f2')
plt.plot(t_raw, f1_raw, 'b.', linewidth=1, label='raw f1')
#plt.plot(t, (f1 + f2) / 2, 'g-', linewidth=2, label='mean')
plt.xlabel('Time [sec]')
plt.grid()
plt.legend()
plt.show()

In [ ]:
f1_filtered = filt.filter(f1)
f2_filtered = filt.filter(f2)

plt.figure(figsize=(20,15))
# plt.plot(t, f1, 'b-', linewidth=1, label='f1')
# plt.plot(t, f1_filtered, 'r-', linew...idth=2, label='f1_filtered')
plt.plot(t, f2, 'g-', linewidth=1, label='f2')
plt.plot(t, f2_filtered, 'm-', linewidth=2, label='f2_filtered')
# plt.plot(t, (f1 + f2) / 2, 'g-', linewidth=2, label='mean')
plt.xlabel('Time [sec]')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# save labels in csv format (after filtering)
for subject_number in range(1, 2):
    for experiment_number in range(1, 11):
        output_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/'.format(subject_number, experiment_number)
        frames_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
        force_data_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/raw_force_data.txt'.format(subject_number, experiment_number)
        t, f1, f2 = get_force_data(frames_path, force_data_path)
        f1_filtered = filt.filter(f1)
        f2_filtered = filt.filter(f2)
        labels = np.concatenate((f1_filtered[..., np.newaxis], f2_filtered[..., np.newaxis]), axis=1)
        np.savetxt('{}labels.csv'.format(output_path), labels, delimiter=",")

In [ ]:
# visually check if data corresponds
def play_with_force(image_path, labels_path):
    global labels
#     cap = cv2.VideoCapture(video_file_path)
    
    labels = np.loadtxt(labels_path, delimiter=',')[:,1:]
    labels = np.mean(labels, axis=1)
    max_label = np.max(labels)

    x1 = 1700
    x2 = x1 + 100
    y2 = 900
    y1 = y2 - 100
    
    n = 0
    while(True):
        frame = cv2.imread('{}{:04d}.png'.format(image_path, n))

        k = labels[n] * 255.0 / max_label
        y1 = y2 - int(800 * labels[n] / max_label)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, k), thickness=-1)
        n += 1

        # Display the resulting frame
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cv2.destroyAllWindows()

In [ ]:
subject_number = 1
experiment_number = 2
image_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/frames/'.format(subject_number, experiment_number)
labels_path = '/media/viktor/Samsung_T5/Research/dataset/{:02d}/{:02d}/labels.csv'.format(subject_number, experiment_number)
play_with_force(image_path, labels_path)